# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information.

In this project we will deal with the following questions: 

Q1. Which country has the largest and smallest population? How is population growth in these countries?  
Q2. Which country has more waters than land?  
Q3. Which are the top 3 contries that have a higher death rate than birth rate?  

![mapa mundi](https://images.educamaisbrasil.com.br/content/banco_de_imagens/guia-de-estudo/D/mapa-mundi-reproducao.jpg)

## Loading the data

In [1]:
# installing ipython-sql to the jupyter notebook
# !conda install -yc conda-forge ipython-sql

Connecting the Jupyter Notebook to the database file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [3]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


 Here are the descriptions for some of the columns:

- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the average annual percent change in the population, resulting from a surplus (or deficit) of births over deaths and the balance of migrants entering and leaving a country.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

### Q1. Which country has the largest and smallest population? How is population growth in these countries?

Look for any outlier country

In [13]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth), ROUND(AVG(population_growth), 2)
    FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth),"ROUND(AVG(population_growth), 2)"
0,7256490011,0.0,4.02,1.2


Select countries with the largest and smallest population.

In [9]:
%%sql
SELECT name, population, population_growth
    FROM facts
    WHERE population IN ((SELECT MIN(population) FROM facts), 
                         (SELECT MAX(population) FROM facts));

 * sqlite:///factbook.db
Done.


name,population,population_growth
Antarctica,0,None
World,7256490011,1.08


It makes sense since Antarctica has no permanent human inhabitants as we can see [here](https://en.wikipedia.org/wiki/Antarctica#Population) (acessed in 01/09/2020).

<img alt="Antarctica" src="https://www.thetimes.co.uk/expert-traveller/wp-content/uploads/2019/01/GettyImages-580748015-2200x880.jpg" width="600" height="400">

Only the entire world can have 7256490011 inhabitants. Next, we recalculate the the countries with tha largest and the smallest population, excludind the rows for world and Antarctica.

In [10]:
%%sql
SELECT name, population, population_growth
    FROM facts
    WHERE population IN ((SELECT MIN(population) FROM facts WHERE name != 'Antarctica'), 
                         (SELECT MAX(population) FROM facts WHERE name != 'World'));

 * sqlite:///factbook.db
Done.


name,population,population_growth
China,1367485388,0.45
Pitcairn Islands,48,0.0


"With a population of only around fifty, the people of Pitcairn are descended from the mutineers of HMAV Bounty and their Tahitian companions. Pitcairn Island is approximately 3.2km (2 miles) long and 1.6km (1 mile) wide with the capital Adamstown located above Bounty Bay and accessed by the aptly named road, *The Hill of Difficulty*." (Text extracted from [https://www.government.pn/](https://www.government.pn/) acessed in 01/09/2021.)

<img src=https://www.coralexpeditions.com/wp-content/uploads/2020/02/Pitcairn-Island.jpg alt='Pitcairn Islands' width="600" height="400">

"The islands have suffered a substantial population decline since 1940, and the viability of the island's community is in doubt (see Potential extinction, [here](https://en.wikipedia.org/wiki/Pitcairn_Islands#Potential_extinction)). The government has tried to attract migrants. However, these initiatives have not been effective." (Text extracted from https://en.wikipedia.org/wiki/Pitcairn_Islands acessed in 01/09/2021.)

<img src=https://www.melhoresdestinos.com.br/wp-content/uploads/2018/01/rua-qianmen-pequim-china-e1515286964915.jpg alt="Pequim, China" width="600" height="400">

China is the most populated country in the world. But the population grouth 0.45 is under the world mean 1.2.

### Q2. Which country has more waters than land?

We are interested in countries such that the waterarea is bigger than the land area. In other words, countries such that waterarea - land area greater than zero.

In [37]:
%%sql
SELECT name, area_land,	area_water, ROUND(CAST(area_water AS FLOAT) / CAST(area AS FLOAT), 3) AS water_land_ratio
    FROM facts
    WHERE (area_water IS NOT NULL AND area_land IS NOT NULL)
    AND area_water -  area_land > 0;

 * sqlite:///factbook.db
Done.


name,area_land,area_water,water_land_ratio
British Indian Ocean Territory,60,54340,0.999
Virgin Islands,346,1564,0.819


<img src=https://www.banknotes.com/chagos.jpg alt='British Indian Ocean Territory'>

British Indian Ocean Territory is the country with the biggest waterland ratio (almost 100% of it's area is water). It is a British overseas territory located in the middle of the Indian Ocean, approximately halfway between Africa and Indonesia.

### Q3. Which are the top 3 contries that have a higher death rate than birth rate?

The country with the largest spread between birth and dead rates is the one with the biggest value in the expression:  
<div align="center">death_rate - birth_rate</div>

In [42]:
%%sql
SELECT name, birth_rate, death_rate, ROUND(death_rate - birth_rate, 2) AS 'death_minus_birth_rates' 
    FROM facts
    WHERE (birth_rate IS NOT NULL AND death_rate IS NOT NULL)
    ORDER BY death_minus_birth_rates DESC
    LIMIT 3;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate,death_minus_birth_rates
Bulgaria,8.92,14.44,5.52
Serbia,9.08,13.66,4.58
Latvia,10.0,14.31,4.31


<img src=https://panoramacrypto.com.br/wp-content/uploads/2019/08/66492979_m-690x340.jpg alt='Bulgaria'>

Falling birth rates are common in most countries in Europe.  

"In 2018 the average total fertility rate (TFR) across Bulgaria was 1.56 children per woman, below the replacement rate of 2.1, it remains considerably below the high of 5.83 children per woman in 1905. Bulgaria subsequently has one of the oldest populations in the world, with the average age of 43 years."
(Text extracted from https://en.wikipedia.org/wiki/Bulgaria#Demographics acessed in 01/09/2021.)

"High death rates result from a combination of an ageing population, a high number of people at risk of poverty and a weak healthcare system. More than 80% of all deaths are due to cancer and cardiovascular conditions; nearly a fifth of those are avoidable."
(Text extracted from https://en.wikipedia.org/wiki/Bulgaria#Demographics acessed in 01/09/2021.)